In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from rapidfuzz import process 

df = pd.read_csv('laptop_price.csv', encoding='latin-1')


df['Ram'] = df['Ram'].str.replace('GB', '').str.strip()
df['Ram'] = pd.to_numeric(df['Ram'], errors='coerce')


def extract_memory_types(mem_str):
    mem_str = str(mem_str).lower()
    ssd = 0
    hdd = 0
    if 'ssd' in mem_str:
        ssd_part = mem_str.split('ssd')[0]
        ssd = int(''.join(filter(str.isdigit, ssd_part)))
    if 'hdd' in mem_str:
        if '+' in mem_str:
            hdd_part = mem_str.split('+')[-1]
        else:
            hdd_part = mem_str
        hdd = int(''.join(filter(str.isdigit, hdd_part)))
    return pd.Series([ssd, hdd])

df[['SSD', 'HDD']] = df['Memory'].apply(extract_memory_types)


comp = ['Apple', 'HP', 'Acer', 'Asus', 'Dell', 'Lenovo', 'Chuwi', 'MSI',
        'Microsoft', 'Toshiba', 'Huawei', 'Xiaomi', 'Vero', 'Razer',
        'Mediacom', 'Samsung', 'Google', 'Fujitsu', 'LG']

res = ['IPS Panel Retina Display 2560x1600', '1440x900',
       'Full HD 1920x1080', 'IPS Panel Retina Display 2880x1800',
       '1366x768', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Retina Display 2304x1440',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'Full HD / Touchscreen 1920x1080',
       'Touchscreen / Quad HD+ 3200x1800',
       'IPS Panel Touchscreen 1920x1200', 'Touchscreen 2256x1504',
       'Quad HD+ / Touchscreen 3200x1800', 'IPS Panel 1366x768',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x2160',
       'IPS Panel Full HD 2160x1440', '4K Ultra HD / Touchscreen 3840x2160',
       'Touchscreen 2560x1440', '1600x900', 'IPS Panel 4K Ultra HD 3840x2160',
       '4K Ultra HD 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Full HD 1366x768', 'IPS Panel 2560x1440',
       'IPS Panel Full HD 2560x1440', 'IPS Panel Retina Display 2736x1824',
       'Touchscreen 2400x1600', '2560x1440', 'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel Quad HD+ 3200x1800', 'IPS Panel Quad HD+ / Touchscreen 3200x1800',
       'IPS Panel Touchscreen 1366x768', '1920x1080',
       'IPS Panel Full HD 1920x1200', 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2560x1440', 'Touchscreen / Full HD 1920x1080',
       'Quad HD+ 3200x1800', 'Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2400x1600']




company_encoder = OrdinalEncoder(categories=[comp], dtype=int)
screenres_encoder = OrdinalEncoder(categories=[res], dtype=int)
product_encoder = LabelEncoder()


df['Company_enc'] = company_encoder.fit_transform(df[['Company']])
df['ScreenRes_enc'] = screenres_encoder.fit_transform(df[['ScreenResolution']])
df['Prod_enc'] = product_encoder.fit_transform(df['Product'])

df['Price_in_Naira'] = df['Price_in_euros'] * 1600

X = df[['Ram', 'SSD', 'HDD', 'Company_enc', 'Prod_enc', 'ScreenRes_enc']]
y = df['Price_in_Naira']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)


def fuzzy_correct(user_input, valid_options, field_name='field', cutoff=70):
    match = process.extractOne(user_input, valid_options, score_cutoff=cutoff)
    if match:
        return match[0]
    else:
        raise ValueError(f"No close match found for {field_name}: '{user_input}'.")


print("\nExample input:\n - 8GB RAM, 512GB SSD, Dell, XPS 13, Full HD 1920x1080")
user_input = input("Enter Laptop Specifications: ")


try:
    parts = [x.strip() for x in user_input.split(',')]

    
    ram_str = parts[0]
    ram_input = int(ram_str.lower().replace('gb ram', '').strip())

   
    ssd_input = 0
    hdd_input = 0

    if 'ssd' in parts[1].lower():
        ssd_str = parts[1]
        ssd_input = int(ssd_str.lower().replace('gb ssd', ''))
        
        company_input, product_input, screenres_input = parts[2], parts[3], parts[4]
    elif 'hdd' in parts[1].lower():
        hdd_str = parts[1]
        hdd_input = int(hdd_str.lower().replace('gb hdd', '').replace('tb hdd', '000').strip())
        
        company_input, product_input, screenres_input = parts[2], parts[3], parts[4]
    else:
        raise ValueError("Second input must specify SSD or HDD.")

    
    company_corrected = fuzzy_correct(company_input, comp, field_name='Company')
    screenres_corrected = fuzzy_correct(screenres_input, res, field_name='ScreenResolution')

    
    company_encoded = company_encoder.transform([[company_corrected]])[0][0]
    screenres_encoded = screenres_encoder.transform([[screenres_corrected]])[0][0]

    
    product_list = list(product_encoder.classes_)
    product_corrected = fuzzy_correct(product_input, product_list, field_name='Product', cutoff=60)

  
    product_encoded = product_encoder.transform([product_corrected])[0]

   
    user_features = [[ram_input, ssd_input, hdd_input, company_encoded, product_encoded, screenres_encoded]]
    
    
    predicted_price = model.predict(user_features)
    print(f"\n💻 Predicted Laptop Price: N{predicted_price[0]:.2f}")

except Exception:
    print("\n⚠️ Error! Please check your input carefully and enter valid system specifications.")
   



Example input:
 - 8GB RAM, 512GB SSD, Dell, XPS 13, Full HD 1920x1080


Enter Laptop Specifications:  8GB RAM, 512GB SSD, Dell, XPS 13, Full HD 1920x1080



💻 Predicted Laptop Price: N2290760.00


C:\Users\adele\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
C:\Users\adele\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
C:\Users\adele\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [19]:
df


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros,SSD,HDD,Company_enc,ScreenRes_enc,Prod_enc,Price_in_Naira
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69,128,0,0,0,300,2143504.0
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94,0,0,0,1,301,1438304.0
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,256,0,1,2,50,920000.0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45,512,0,0,3,300,4059920.0
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,256,0,0,0,300,2885760.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00,128,0,5,7,580,1020800.0
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00,512,0,5,30,588,2398400.0
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00,0,0,5,4,196,366400.0
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00,0,1,1,4,2,1222400.0


0       2143504.0
1       1438304.0
2        920000.0
3       4059920.0
4       2885760.0
          ...    
1298    1020800.0
1299    2398400.0
1300     366400.0
1301    1222400.0
1302     590400.0
Name: Price_in_euros, Length: 1303, dtype: float64